In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoConfig

In [166]:
MODEL_CHECKPOINT = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)
text = "time flies like an arrow"

In [204]:
VOCAB_SIZE = 30522
HIDDEN_SIZE = 768
MAX_POSITION = 512
NUM_HEADS = 12
INTERMEDIATE_SIZE = 3072
DROPOUT_P = 0.1

In [167]:
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [196]:
class Embeddings(nn.Module):
    def __init__(self, vocab_size, embed_dim, max_position):
        super().__init__()
        self.token_embeddings = nn.Embedding(vocab_size, embed_dim)
        self.position_embeddings = nn.Embedding(max_position, embed_dim)
        self.layernorm = nn.LayerNorm(embed_dim, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        seq_len = input_ids.size(-1)
        position_ids = torch.arange(seq_len, dtype=torch.long).view(1, -1)

        token_embed = self.token_embeddings(input_ids)
        position_embed = self.position_embeddings(position_ids)

        embeddings = token_embed + position_embed
        embeddings = self.layernorm(embeddings)
        return self.dropout(embeddings)

class Attention(nn.Module):
    def __init__(self, embed_dim, head_dim, mask=None):
        super().__init__()
        self.mask = mask
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)
    
    def forward(self, x):
        q = self.q(x); k = self.k(x); v = self.v(x)
        d_k = q.size(-1) # head_dim
        scores = torch.bmm(q, k.transpose(1, 2)) / d_k ** 0.5

        if self.mask is not None:
            scores = scores.masked_fill(self.mask==0, -float("inf"))

        weights = F.softmax(scores, dim=-1)
        return torch.bmm(weights, v)

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, mask=None):
        super().__init__()
        self.head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList([
            Attention(embed_dim, self.head_dim, mask) for _ in range(num_heads)
        ])
        self.out_fc = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.out_fc(x)

class FeedForward(nn.Module):
    def __init__(self, embed_dim, hidden_size, dropout_p):
        super().__init__()
        self.linear_in = nn.Linear(embed_dim, hidden_size)
        self.linear_out = nn.Linear(hidden_size, embed_dim)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x):
        x = self.linear_in(x)
        x = self.gelu(x)
        x = self.linear_out(x)
        return self.dropout(x)

class EncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, intermediate_size, dropout_p):
        super().__init__()
        self.layernorm_1 = nn.LayerNorm(embed_dim)
        self.layernorm_2 = nn.LayerNorm(embed_dim)
        self.mha = MultiHeadAttention(embed_dim, num_heads)
        self.ff = FeedForward(embed_dim, intermediate_size, dropout_p)

    def forward(self, x):
        x = x + self.mha(self.layernorm_1(x))
        x = x + self.ff(self.layernorm_2(x))
        return x

class DecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, intermediate_size, dropout_p):
        super().__init__()
        self.layernorm_1 = nn.LayerNorm(embed_dim)
        self.layernorm_2 = nn.LayerNorm(embed_dim)
        self.layernorm_3 = nn.LayerNorm(embed_dim)
        self.mask = torch.tril(torch.ones(5, 5))
        self.mha_1 = MultiHeadAttention(embed_dim, num_heads, mask=self.mask)
        self.mha_2 = MultiHeadAttention(embed_dim, num_heads, mask=self.mask)
        self.ff = FeedForward(embed_dim, intermediate_size, dropout_p)

    def forward(self, x):
        x = x + self.mha_1(self.layernorm_1(x))
        x = x + self.mha_2(self.layernorm_2(x))
        x = x + self.ff(self.layernorm_3(x))
        return x



In [197]:
embed = Embeddings(config.vocab_size, config.hidden_size, config.max_position_embeddings)
inputs_embedded = embed(inputs.input_ids)
inputs_embedded.shape

torch.Size([1, 5, 768])

In [198]:
enc = EncoderBlock(config.hidden_size,
                   config.num_attention_heads, config.intermediate_size, config.hidden_dropout_prob)
enc_result = enc(inputs_embedded)
enc_result.shape


torch.Size([1, 5, 768])

In [201]:
denc = DecoderBlock(config.hidden_size, config.num_attention_heads, config.intermediate_size, config.hidden_dropout_prob)
out = denc(inputs_embedded)

out.shape

torch.Size([1, 5, 768])

In [175]:
mask.shape

torch.Size([768, 768])

In [38]:
data = torch.randn(4,2)
q = nn.Linear(2, 5)
q(data).size(-1)


5

In [19]:
4 ** 0.5

2.0

In [94]:
import numpy as np

In [97]:
np.log(0)

C:\Users\user\AppData\Local\Temp\ipykernel_25016\2933082444.py:1: RuntimeWarning: divide by zero encountered in log
  np.log(0)


-inf